In [1]:
from selenium import webdriver
import time
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
from tensorflow.keras.models import load_model
import cv2
import os
from glob import glob
import math
import tqdm
import tensorflowjs as tfjs
import numpy
import pandas
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import sys
from tensorflow.keras.applications import EfficientNetB0
import efficientnet.keras as efn
import re
import time
import ffmpeg
import youtube_dl

In [17]:
model=load_model(r'C:\Users\kashi\Downloads\proj\NPDIv3fullofull.12-0.17.h5')

In [12]:
driver=webdriver.Chrome(executable_path=r"C:\Users\kashi\go\chromedriver.exe")
driver.maximize_window()

In [13]:
url=driver.current_url
print(url)

https://www.youtube.com/watch?v=_8ZVl2x-euA


In [15]:
ydl_opts = {'outtmpl': r'C:\Users\kashi\Downloads\proj\vid\inpt.mp4',}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] _8ZVl2x-euA: Downloading webpage
[download] Destination: C:\Users\kashi\Downloads\proj\vid\inpt.mp4
[download] 100% of 13.22MiB in 00:04                   


In [19]:
predictions=[]
filecount=0

In [20]:
for file in tqdm.tqdm(os.listdir(r"C:\Users\kashi\Downloads\proj\vid")):

    print(file)
    
    count = 0
    cap = cv2.VideoCapture(r'C:\Users\kashi\Downloads\proj\vid\\'+file)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    print(frameRate)
    #x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename =r"C:\Users\kashi\Downloads\proj\temp\\" + "frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob(r"C:\Users\kashi\Downloads\proj\temp\*.jpg")
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = numpy.array(img)
        img = img/255
        prediction_images.append(img)

    prediction_images=numpy.array(prediction_images)
    pred=model.predict(prediction_images)
    y_pred = numpy.argmax(pred, axis=-1)
   
    for i in range(0,len(y_pred)):
        if y_pred[i]==1 and pred[i][1]<=0.95:
            y_pred[i]=0
    print(numpy.sum(y_pred),math.ceil(10*len(prediction_images)/100))
    confiporn=[]
    confinonporn=[]
    for i in range(0,len(y_pred)):
    	if y_pred[i]==1:
            
    		confiporn.append(pred[i][1])
    	else:
    		confinonporn.append(pred[i][0])
    confiporn=numpy.array(confiporn)
    confinonporn=numpy.array(confinonporn)

    
    if(numpy.sum(y_pred)>=math.ceil(10*len(prediction_images)/100)):
    	predictions.append(1)

    	confidence=numpy.mean(confiporn)
    else:
    	predictions.append(0)
    	confidence=numpy.mean(confinonporn)
    confi = numpy.round(confidence*100)
    filecount+=1
    


  0%|          | 0/1 [00:00<?, ?it/s]

inpt.mp4
25.0


100%|██████████| 1/1 [00:36<00:00, 36.63s/it]

0 13


In [21]:
print(predictions)

[0]


In [ ]:
predictions=numpy.array(predictions)
actual=numpy.zeros(filecount)
print(accuracy_score(predictions,actual))

In [22]:
if predictions[0]==0:
    driver.execute_script("document.getElementById('player').style.filter='blur(100px)';")